In [1]:
import findspark
findspark.init('/opt/spark')  # Adjust the path if Spark is installed elsewhere

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("example").getOrCreate()

24/05/22 21:53:41 WARN Utils: Your hostname, chuot-HP-Pavilion-Laptop-14-ce3xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlo1)
24/05/22 21:53:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/22 21:53:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/22 21:53:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [16]:
# Define the path to your JSON file
from pyspark.sql.functions import col, udf, to_date, when, regexp_extract, explode, col, regexp_replace, to_timestamp, concat, lit
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, ArrayType
import json
import re
json_file_path = 'vieclam24h.json'
# Read the JSON file into a DataFrame
df = spark.read.option("multiline","true").json(json_file_path)

# df_exploded = df.select(explode(col("category")).alias("unique_category"))

# unique_categories = df_exploded.select("unique_category").distinct().collect()
# print(len(unique_categories))
# for row in unique_categories:
#     pass
#     print(row.unique_category)
# unique_experience_df = df.select("experience").distinct()

# unique_experience_df.show()

# from pyspark.sql.functions import col
unique_values = df.select(col("role")).distinct().collect()

# In ra các giá trị duy nhất
for row in unique_values:
    print(row.role)





#rdd.toDF().select("salary").show(1)

# Salary


Chưa cập nhật
Chuyên viên- nhân viên
Quản lý cấp cao
Cộng tác viên
Chuyên gia
Quản lý nhóm- giám sát
Quản lý cấp trung


In [ ]:
# Salary
def parse_salary(salary):
    if salary is None or salary.lower() == "thỏa thuận":
        return json.dumps({"type": 2, "min": None, "max": None, "fixed_value": None})
    elif "triệu" in salary:
        salary = salary.replace(" triệu", "").replace(",", ".").strip()
        if "-" in salary:
            min_salary, max_salary = salary.split("-")
            return json.dumps({"type": 1, "min": float(min_salary), "max": float(max_salary), "fixed_value": None})
        else:
            return json.dumps({"type": 0, "min": None, "max": None, "fixed_value": float(salary)})
    return json.dumps({"type": 2, "min": None, "max": None, "fixed_value": None})

parse_salary_udf = udf(parse_salary, StringType())
df = df.withColumn("salary", parse_salary_udf(col("salary")))
rdd = df.rdd.map(lambda row: row.asDict())

In [34]:
# Experience
def transform_experience(df):
    df = df.withColumn("experience", when(col("experience") == "Chưa có kinh nghiệm", 0).otherwise(col("experience")))
    
    df = df.withColumn("experience", regexp_replace(col("experience"), "(Dưới |Hơn |năm)", ""))
    return df

transformed_df = transform_experience(df)

# Hiển thị kết quả
transformed_df.select("experience").show()

+----------+
|experience|
+----------+
|        2 |
|        1 |
|        1 |
|        1 |
|         0|
|         0|
|        2 |
|        1 |
|        1 |
|         0|
|         0|
|        1 |
|        1 |
|         0|
|         0|
|        1 |
|        1 |
|         0|
|        2 |
|         0|
+----------+
only showing top 20 rows



In [64]:
# Type
# 0. Toàn thời gian cố định
# 1. Bán thời gian cố định
# 2. Toàn thời gian tạm thời
# 3. Khác
df = df.withColumn(
    "type",
     when(col("type") == "Toàn thời gian cố định", 0)
    .when(col("type") == "Bán thời gian cố định", 1)
    .when(col("type") == "Toàn thời gian tạm thời", 2)
    .when(col("type") == "Khác", 3)
    .otherwise(None)
)
df.select("type").show(5)


+----+
|type|
+----+
|   0|
|   0|
|   0|
|   0|
|   0|
+----+
only showing top 5 rows



In [3]:
# Expiration
df = df.withColumn("expiration", to_timestamp(concat(col("expiration"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))
df.select("expiration").show(5)

+-------------------+
|         expiration|
+-------------------+
|2024-06-15 00:00:00|
|2024-06-15 00:00:00|
|2024-05-31 00:00:00|
|2024-06-15 00:00:00|
|2024-05-21 00:00:00|
+-------------------+
only showing top 5 rows



In [12]:
# Description
def clean_description(description):
    return [re.sub(r'^[\-\•\d\.\s]+', '', desc).strip() for desc in description]
clean_description_udf = udf(clean_description, ArrayType(StringType()))
df = df.withColumn("description", clean_description_udf(col("description")))
df.select("description").show(5)


+--------------------+
|         description|
+--------------------+
|[Sửa chữa, bảo dư...|
|[Trực tiếp kiểm t...|
|[Thi công, lắp đặ...|
|[Nhận và xử lý cá...|
|[Tìm kiếm khách h...|
+--------------------+
only showing top 5 rows



In [17]:
# Role
def classify_role(role):
    if role == "Chuyên viên- nhân viên":
        return 1
    elif role == "Quản lý cấp cao":
        return 2
    elif role == "Cộng tác viên":
        return 3
    elif role == "Chuyên gia":
        return 4
    elif role == "Quản lý nhóm- giám sát":
        return 5
    elif role == "Quản lý cấp trung":
        return 6
    else:
        return 0
classify_role_udf = udf(classify_role, IntegerType())
df = df.withColumn("role", classify_role_udf(col("role")))

df.select("role").show(50)


+----+
|role|
+----+
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   5|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   2|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   5|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
+----+
only showing top 50 rows

